In [29]:
!pip install flask flask-ngrok
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip


zsh:1: command not found: wget
unzip:  cannot find or open ngrok-stable-linux-amd64.zip, ngrok-stable-linux-amd64.zip.zip or ngrok-stable-linux-amd64.zip.ZIP.


In [31]:
!lsof -i :5000


COMMAND    PID        USER   FD   TYPE             DEVICE SIZE/OFF NODE NAME
ControlCe 1527 sangammalik   10u  IPv4  0x33608be604452b4      0t0  TCP *:commplex-main (LISTEN)
ControlCe 1527 sangammalik   11u  IPv6 0x15cbc4e87280ba68      0t0  TCP *:commplex-main (LISTEN)


In [33]:
!kill -9 <PID>


zsh:1: parse error near `>'


In [35]:
!pip install tensorflow

In [36]:
!pip install pyngrok


In [38]:
!pip install flask flask-ngrok


In [39]:
!pkill ngrok

In [1]:
from pyngrok import ngrok

# Paste your authtoken below
ngrok.set_auth_token("2vlQSYKB7B779ow7hPPFGLxEaIX_7x5wcp3m9oP8LoJ2t7TLP")


In [44]:
!pip install keras

In [3]:
 %%writefile app.py

from flask import Flask, render_template_string, request, flash, render_template
import numpy as np
import os
import cv2
from keras.models import load_model
from pyngrok import ngrok

app = Flask(__name__)
app.secret_key = 'emotion_detection_app'

UPLOAD_FOLDER = 'uploads'
os.makedirs(UPLOAD_FOLDER, exist_ok=True)
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER

# Load the model
try:
    model = load_model('custom_cnn_model.h5')
    model_loaded = True
except Exception as e:
    print(f"Error loading model: {e}")
    model_loaded = False

# Emotion labels based on your model training
emotion_labels = ['Angry', 'Disgust', 'Fear', 'Happy', 'Neutral', 'Sad', 'Surprise']

@app.route('/', methods=['GET', 'POST'])
def index():
    prediction = None
    if request.method == 'POST':
        print("[DEBUG] POST request received")

        if not model_loaded:
            print("[ERROR] Model not loaded")
            flash("Model not loaded. Please ensure 'custom_cnn_model.h5' exists.")
            return render_template('index.html', prediction=None, error=True)

        if 'image' not in request.files:
            print("[ERROR] No image in request")
            flash('No file part')
            return render_template('index.html', prediction=None, error=True)

        file = request.files['image']
        if file.filename == '':
            print("[ERROR] No selected file")
            flash('No selected file')
            return render_template('index.html', prediction=None, error=True)

        if file:
            filepath = os.path.join(app.config['UPLOAD_FOLDER'], file.filename)
            file.save(filepath)
            print(f"[INFO] File saved to {filepath}")

            try:
                # Load image and detect face
                img = cv2.imread(filepath)
                gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

                face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
                faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)

                if len(faces) == 0:
                    print("[ERROR] No face detected in the image.")
                    flash('No face detected in the uploaded image.')
                    return render_template('index.html', prediction=None, error=True)

                # Use the first detected face
                (x, y, w, h) = faces[0]
                face = gray[y:y+h, x:x+w]
                face = cv2.resize(face, (48, 48))
                face = face.astype('float32') / 255.0
                face = np.reshape(face, (1, 48, 48, 1))

                prediction_idx = np.argmax(model.predict(face))
                prediction = emotion_labels[prediction_idx]
                print(f"[SUCCESS] Predicted Emotion: {prediction}")
                
            except Exception as e:
                print(f"[ERROR] Exception during processing: {str(e)}")
                flash(f'Error processing image: {str(e)}')
                return render_template('index.html', prediction=None, error=True)

    return render_template('index.html', prediction=prediction, error=False)

if __name__ == '__main__':
    if model_loaded:
        print("Model loaded successfully. Starting application...")
    else:
        print("WARNING: Model not loaded. Place 'custom_cnn_model.h5' in the root directory.")

    # Start ngrok tunnel
    public_url = ngrok.connect(5001)
    print(f" * ngrok tunnel available at: {public_url}")

    app.run(port=5001)



Overwriting app.py


In [5]:
# %%writefile app.py

# from flask import Flask, render_template, request, flash
# import numpy as np
# import os
# import cv2
# from keras.models import load_model
# from pyngrok import ngrok

# app = Flask(__name__)
# app.secret_key = 'emotion_detection_app'

# UPLOAD_FOLDER = 'uploads'
# os.makedirs(UPLOAD_FOLDER, exist_ok=True)
# app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER

# # Load the model
# try:
#     model = load_model('model.h5')
#     model_loaded = True
# except Exception as e:
#     print(f"Error loading model: {e}")
#     model_loaded = False

# # Emotion labels based on model training
# emotion_labels = ['Angry', 'Disgust', 'Fear', 'Happy', 'Neutral', 'Sad', 'Surprise']

# @app.route('/', methods=['GET', 'POST'])
# def index():
#     prediction = None
#     error = False

#     if request.method == 'POST':
#         print("[DEBUG] POST request received")

#         if not model_loaded:
#             print("[ERROR] Model not loaded")
#             flash("Model not loaded. Please ensure 'model.h5' exists.")
#             return render_template('index.html', prediction=None, error=True)

#         file = request.files.get('image')
#         if not file or file.filename == '':
#             print("[ERROR] No selected file")
#             flash('No file selected')
#             return render_template('index.html', prediction=None, error=True)

#         filepath = os.path.join(app.config['UPLOAD_FOLDER'], file.filename)
#         file.save(filepath)
#         print(f"[INFO] File saved to {filepath}")

#         try:
#             # Load and preprocess the image
#             img = cv2.imread(filepath)
#             gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

#             # face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
#             # faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)

#             def detect_face_dnn(img):
#                 modelFile = "res10_300x300_ssd_iter_140000_fp16.caffemodel"
#                 configFile = "deploy.prototxt"
#                 net = cv2.dnn.readNetFromCaffe(configFile, modelFile)

#                 h, w = img.shape[:2]
#                 blob = cv2.dnn.blobFromImage(cv2.resize(img, (300, 300)), 1.0, (300, 300), (104.0, 177.0, 123.0))
#                 net.setInput(blob)
#                 detections = net.forward()

#                 faces = []
#                 for i in range(detections.shape[2]):
#                     confidence = detections[0, 0, i, 2]
#                     if confidence > 0.5:
#                         box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
#                         (x1, y1, x2, y2) = box.astype("int")
#                         faces.append((x1, y1, x2 - x1, y2 - y1))
#                 return faces

#             faces = detect_face_dnn(img)


#             if len(faces) == 0:
#                 print("[ERROR] No face detected in the image.")
#                 flash('No face detected in the uploaded image.')
#                 return render_template('index.html', prediction=None, error=True)

#             for (x, y, w, h) in faces:
#                 face = gray[y:y+h, x:x+w]
#                 face = cv2.resize(face, (48, 48))
#                 face = face.astype('float32') / 255.0
#                 face = np.expand_dims(face, axis=-1)  # (48, 48, 1)
#                 face = np.expand_dims(face, axis=0)   # (1, 48, 48, 1)

#                 preds = model.predict(face)
#                 prediction_idx = np.argmax(preds)
#                 prediction = emotion_labels[prediction_idx]
#                 confidence = np.max(preds)
#                 print(f"[SUCCESS] Predicted Emotion: {prediction} (Confidence: {confidence:.2f})")
#                 break  # Only use first face

#         except Exception as e:
#             print(f"[ERROR] Exception during processing: {str(e)}")
#             flash(f'Error processing image: {str(e)}')
#             error = True

#     return render_template('index.html', prediction=prediction, error=error)

# if __name__ == '__main__':
#     if model_loaded:
#         print("Model loaded successfully. Starting application...")
#     else:
#         print("WARNING: Model not loaded. Place 'model.h5' in the root directory.")

#     public_url = ngrok.connect(5001)
#     print(f" * ngrok tunnel available at: {public_url}")
#     app.run(port=5001)


In [7]:
# %%writefile app.py

# from flask import Flask, render_template, request, flash
# import numpy as np
# import os
# import cv2
# from keras.models import load_model
# from pyngrok import ngrok

# app = Flask(__name__)
# app.secret_key = 'emotion_detection_app'

# UPLOAD_FOLDER = 'uploads'
# os.makedirs(UPLOAD_FOLDER, exist_ok=True)
# app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER

# # Load the model
# try:
#     model = load_model('model.h5')
#     model_loaded = True
# except Exception as e:
#     print(f"[ERROR] Loading model: {e}")
#     model_loaded = False

# emotion_labels = ['Angry', 'Disgust', 'Fear', 'Happy', 'Neutral', 'Sad', 'Surprise']

# # --- DNN Face Detection ---
# def detect_face_dnn(img):
#     try:
#         model_path = os.path.join(os.path.dirname(__file__), "res10_300x300_ssd_iter_140000.caffemodel")
#         config_path = os.path.join(os.path.dirname(__file__), "deploy.prototxt")
#         net = cv2.dnn.readNetFromCaffe(config_path, model_path)

#         h, w = img.shape[:2]
#         blob = cv2.dnn.blobFromImage(cv2.resize(img, (300, 300)), 1.0,
#                                      (300, 300), (104.0, 177.0, 123.0))
#         net.setInput(blob)
#         detections = net.forward()

#         faces = []
#         for i in range(detections.shape[2]):
#             confidence = detections[0, 0, i, 2]
#             if confidence > 0.5:
#                 box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
#                 (x1, y1, x2, y2) = box.astype("int")
#                 faces.append((x1, y1, x2 - x1, y2 - y1))
#         return faces
#     except Exception as e:
#         print(f"[ERROR] DNN face detection failed: {e}")
#         return []

# # --- Flask Route ---
# @app.route('/', methods=['GET', 'POST'])
# def index():
#     prediction = None
#     error = False

#     if request.method == 'POST':
#         file = request.files.get('image')
#         if not file or file.filename == '':
#             flash('No file selected')
#             return render_template('index.html', prediction=None, error=True)

#         filepath = os.path.join(app.config['UPLOAD_FOLDER'], file.filename)
#         file.save(filepath)
#         print(f"[INFO] File saved to {filepath}")

#         try:
#             img = cv2.imread(filepath)
#             gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

#             # Try DNN first
#             faces = detect_face_dnn(img)

#             # Fallback to Haar cascade if DNN fails
#             if not faces:
#                 print("[INFO] Falling back to Haar cascade")
#                 face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
#                 faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)

#             if len(faces) == 0:
#                 flash('No face detected in the uploaded image.')
#                 return render_template('index.html', prediction=None, error=True)

#             # Use the first face detected
#             (x, y, w, h) = faces[0]
#             face = gray[y:y+h, x:x+w]
#             face = cv2.resize(face, (48, 48))
#             face = face.astype('float32') / 255.0
#             face = np.expand_dims(face, axis=-1)  # (48, 48, 1)
#             face = np.expand_dims(face, axis=0)   # (1, 48, 48, 1)

#             # preds = model.predict(face)
#             # prediction_idx = np.argmax(preds)
#             # prediction = emotion_labels[prediction_idx]
#             # confidence = np.max(preds)
#             # print(f"[SUCCESS] Predicted Emotion: {prediction} (Confidence: {confidence:.2f})")

#             preds = model.predict(face)
#             print(f"[DEBUG] Raw model predictions: {preds}")  # Debugging the raw prediction array
#             prediction_idx = np.argmax(preds)
#             prediction = emotion_labels[prediction_idx]
#             confidence = np.max(preds)
#             print(f"[DEBUG] Predicted Emotion: {prediction} (Confidence: {confidence:.2f})")

#         except Exception as e:
#             print(f"[ERROR] Processing image: {str(e)}")
#             flash(f'Error processing image: {str(e)}')
#             error = True

#     return render_template('index.html', prediction=prediction, error=error)

# # --- Run App ---
# if __name__ == '__main__':
#     if model_loaded:
#         print("Model loaded successfully. Starting application...")
#     else:
#         print("WARNING: Model not loaded. Place 'model.h5' in the root directory.")

#     public_url = ngrok.connect(5001)
#     print(f" * ngrok tunnel available at: {public_url}")
#     app.run(port=5001)


In [9]:
!mkdir templates

mkdir: templates: File exists


In [11]:
 %%writefile templates/index.html

<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <title>Emotion Detection</title>
    <style>
        body {
            margin: 0;
            font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
            background: linear-gradient(to right, #6a11cb, #2575fc);
            display: flex;
            justify-content: center;
            align-items: center;
            height: 100vh;
            color: white;
        }

        .container {
            background-color: rgba(0, 0, 0, 0.5);
            padding: 40px;
            border-radius: 15px;
            text-align: center;
            box-shadow: 0 4px 20px rgba(0, 0, 0, 0.4);
        }

        h1 {
            font-size: 2.5rem;
            margin-bottom: 30px;
        }

        input[type="file"] {
            margin: 15px 0;
            padding: 10px;
            background: white;
            border-radius: 8px;
            border: none;
            cursor: pointer;
        }

        input[type="submit"] {
            background-color: #00c9ff;
            color: white;
            padding: 10px 25px;
            border: none;
            border-radius: 8px;
            cursor: pointer;
            font-size: 1rem;
            transition: background-color 0.3s ease;
        }

        input[type="submit"]:hover {
            background-color: #007acc;
        }
    </style>
</head>
<body>
        <div class="container">
        <h1>🎭 Emotion Detector</h1>
        <p>Upload an image to detect the emotion on the face.</p>

        <form id="uploadForm" method="POST" enctype="multipart/form-data">
            <div class="upload-box" id="drop-area">
                <input type="file" name="image" id="imageInput" accept="image/*" required>
                <label for="imageInput" class="upload-label">📤 Click or Drag & Drop an image</label>
                <img id="preview" src="#" alt="Image Preview" style="display: none;" />
            </div>
            <button type="submit">Detect Emotion</button>
        </form>

        {% with messages = get_flashed_messages() %}
            {% if messages %}
                <div class="error-messages">
                    {% for message in messages %}
                        <p class="error">{{ message }}</p>
                    {% endfor %}
                </div>
            {% endif %}
        {% endwith %}

        {% if prediction %}
            <div class="result">
                <h2>Predicted Emotion: <span>{{ prediction }}</span></h2>
            </div>
        {% endif %}
    </div>

    <!-- Optional: add preview image logic -->
    <script>
        document.getElementById('imageInput').addEventListener('change', function (event) {
            const file = event.target.files[0];
            const preview = document.getElementById('preview');

            if (file) {
                const reader = new FileReader();
                reader.onload = function (e) {
                    preview.src = e.target.result;
                    preview.style.display = 'block';
                    preview.style.maxWidth = '300px';
                    preview.style.marginTop = '10px';
                }
                reader.readAsDataURL(file);
            }
        });
    </script>
</body>
</html>






Overwriting templates/index.html


In [13]:
!ngrok config add-authtoken 2vlQSYKB7B779ow7hPPFGLxEaIX_7x5wcp3m9oP8LoJ2t7TLP

Authtoken saved to configuration file: /Users/sangammalik/Library/Application Support/ngrok/ngrok.yml


In [ ]:
!python app.py


Model loaded successfully. Starting application...
 * ngrok tunnel available at: NgrokTunnel: "https://723d-38-183-11-240.ngrok-free.app" -> "http://localhost:5001"
 * Serving Flask app 'app'
 * Debug mode: off
INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5001
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [17/May/2025 08:23:30] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [17/May/2025 08:23:30] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [17/May/2025 08:23:31] "GET /favicon.ico HTTP/1.1" 404 -
[DEBUG] POST request received
[INFO] File saved to uploads/IMG_0061.heic
[ERROR] Exception during processing: OpenCV(4.11.0) /Users/xperience/GHA-Actions-OpenCV/_work/opencv-python/opencv-python/opencv/modules/imgproc/src/color.cpp:199: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'

INFO:werkzeug:127.0.0.1 - - [17/May/2025 08:23